In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from copy import deepcopy

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 2. CNN Model Definition
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        # Automatically calculate the flattened size after pooling
        self.fc1 = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        return x

# Instantiate the model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Training Function
def train_model(model, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 4. Testing Function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# 5. Unlearning Function
def unlearn_data(model, original_data, data_to_unlearn, learning_rate=0.001):
    unlearned_model = deepcopy(model)
    optimizer_unlearn = optim.Adam(unlearned_model.parameters(), lr=learning_rate)

    # DataLoader for unlearning data subset
    unlearn_loader = DataLoader(dataset=Subset(original_data, data_to_unlearn), batch_size=64, shuffle=True)

    unlearned_model.train()
    for images, labels in unlearn_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer_unlearn.zero_grad()
        outputs = unlearned_model(images)
        loss = criterion(outputs, labels)
        loss.backward()  # Reverse gradients to "forget"
        optimizer_unlearn.step()

    return unlearned_model

# 6. Run Training and Evaluation
# Train the model
train_model(model, train_loader, num_epochs=10)
print(f'Accuracy before unlearning: {test_model(model, test_loader):.2f}%')

# Choose a subset of data for unlearning (e.g., first 100 samples)
data_to_unlearn_indices = np.random.choice(len(train_dataset), 100, replace=False)

# Perform unlearning
unlearned_model = unlearn_data(model, train_dataset, data_to_unlearn_indices, learning_rate=0.001)
print(f'Accuracy after unlearning: {test_model(unlearned_model, test_loader):.2f}%')


100%|██████████| 170M/170M [00:02<00:00, 69.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/10], Loss: 1.4635
Epoch [2/10], Loss: 1.2072
Epoch [3/10], Loss: 1.1083
Epoch [4/10], Loss: 1.0497
Epoch [5/10], Loss: 1.0114
Epoch [6/10], Loss: 0.9812
Epoch [7/10], Loss: 0.9610
Epoch [8/10], Loss: 0.9418
Epoch [9/10], Loss: 0.9243
Epoch [10/10], Loss: 0.9091
Accuracy before unlearning: 63.08%
Accuracy after unlearning: 62.83%


The above is the code for machine unlearning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preprocessing with augmentation
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# Load the dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

# Enhanced CNN Model
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Training and testing functions remain similar as before
# Use optim.Adam with a learning rate scheduler like StepLR
# Instantiate the model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Training Function
def train_model(model, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 4. Testing Function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# 5. Unlearning Function
def unlearn_data(model, original_data, data_to_unlearn, learning_rate=0.001):
    unlearned_model = deepcopy(model)
    optimizer_unlearn = optim.Adam(unlearned_model.parameters(), lr=learning_rate)

    # DataLoader for unlearning data subset
    unlearn_loader = DataLoader(dataset=Subset(original_data, data_to_unlearn), batch_size=64, shuffle=True)

    unlearned_model.train()
    for images, labels in unlearn_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer_unlearn.zero_grad()
        outputs = unlearned_model(images)
        loss = criterion(outputs, labels)
        loss.backward()  # Reverse gradients to "forget"
        optimizer_unlearn.step()

    return unlearned_model

# 6. Run Training and Evaluation
# Train the model
train_model(model, train_loader, num_epochs=10)
print(f'Accuracy before unlearning: {test_model(model, test_loader):.2f}%')

# Choose a subset of data for unlearning (e.g., first 100 samples)
data_to_unlearn_indices = np.random.choice(len(train_dataset), 100, replace=False)

# Perform unlearning
unlearned_model = unlearn_data(model, train_dataset, data_to_unlearn_indices, learning_rate=0.001)
print(f'Accuracy after unlearning: {test_model(unlearned_model, test_loader):.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.6295
Epoch [2/10], Loss: 1.3932
Epoch [3/10], Loss: 1.3366
Epoch [4/10], Loss: 1.3058
Epoch [5/10], Loss: 1.2889
Epoch [6/10], Loss: 1.2702
Epoch [7/10], Loss: 1.2584
Epoch [8/10], Loss: 1.2506
Epoch [9/10], Loss: 1.2447
Epoch [10/10], Loss: 1.2346
Accuracy before unlearning: 55.95%
Accuracy after unlearning: 52.15%
